In [2]:
import os
import pickle

import numpy as np
import pandas as pd
import arviz as az
import xarray as xr

import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

import pystan
%load_ext jupyterstan

PROJECT_ROOT_DIR = "."
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images")
STAN_MODEL_PATH = os.path.join(PROJECT_ROOT_DIR, "stan_models")
STAN_DATA_PATH = os.path.join(PROJECT_ROOT_DIR,'data')

## 几个有用的函数
- save_fig:用于保存仿真的图片到文件

- StanModel_cache: 保存编译好的Stan模型

- StanModel_load: 载入保存的Stan模型

In [3]:
def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
    

# 将编译好的模型，存储成pickle，供直接使用
def StanData_cache(var, data_name, **kwargs):
    path = os.path.join(STAN_DATA_PATH, data_name + '.pkl')
    with open(path,'wb') as f:
        pickle.dump(var, f)
    print("DATA cached as:" + data_name +'.pkl')
    
    
def StanData_load(data_name):
    path = os.path.join(STAN_DATA_PATH, data_name + '.pkl')
    try:
        sm = pickle.load(open(path, 'rb'))
    except:
        raise FileNotFoundError
    else:
        print("Using cached StanDATA: " + data_name)
    return sm

# 将编译好的模型，存储成pickle，供直接使用
def StanModel_cache(compiled_model, model_name, **kwargs):
    path = os.path.join(STAN_MODEL_PATH, model_name + '.pkl')
    with open(path,'wb') as f:
        pickle.dump(compiled_model, f)
    print("Model cached as:" + model_name +'.pkl')
    
    
def StanModel_load(model_name):
    path = os.path.join(STAN_MODEL_PATH, model_name + '.pkl')
    try:
        sm = pickle.load(open(path, 'rb'))
    except:
        raise FileNotFoundError
    else:
        print("Using cached StanModel")
    return sm

## stan模型

### 指数相关函数，带测量误差

In [4]:
%%stan exponential_cov_measure_error
functions {
  matrix cov_exponential(matrix distance_between, real sigma, real fai) {
    int N=rows(distance_between);
    matrix[N, N] cov;
    for (i in 1:N){
      for (j in 1:N){
        if (i==j)
          cov[i, j] = 1;
        else
          cov[i,j] = square(sigma) * exp(-1*fai*distance_between[i,j]);
      }
    }
    
    return cov;
  }
}

data {
  int<lower=1> D; //数据组数
  int<lower=1> N; //感知节点个数
  vector[N] recv_x;// 每组数据有N个感知节点
  vector[N] recv_y;// 
  real tran_power[D]; //信号源发送功率
  real tran_x[D]; //信号源坐标
  real tran_y[D];
  vector[N] tran_recv_distance[D];
  vector[N] Y[D]; 
}

transformed data {
  matrix[N, N] distance_between_recv;
  for (i in 1:N){
    for (j in 1:N){
      distance_between_recv[i,j] = sqrt((recv_x[i]-recv_x[j])^2 + (recv_y[i]-recv_y[j])^2);
    }
  }
}

parameters {
  real gama; // 自由空间传播损耗因子
  real fai;  // matern 协方差函数的距离 参数
  real sigma; // 空间方差  
  real sigma_eta; // 测量误差
}

transformed parameters {

}

model {
  matrix[N,N] bigSigma;
  vector[N] i_vec = rep_vector(1.0, N);
  matrix[N, N] unit_matrix = diag_matrix(i_vec);
  
  /********************/
  /*    指定先验分布    */
  /********************/
  gama ~ uniform(1,10);
  fai ~ normal(20, 100);
  sigma ~ inv_gamma(2, 10);
  sigma_eta ~ inv_gamma(2, 10);
 
  bigSigma = cov_exponential(distance_between_recv, sigma, fai) + square(sigma_eta)*unit_matrix;
  for (i in 1:D){
    Y[i] ~ multi_normal(tran_power[i] - 10 * gama * log10(tran_recv_distance[i]), bigSigma); 
  }
   
}

INFO:pystan:COMPILING THE C++ CODE FOR MODEL exponential_cov_measure_error_968f3a417ff91cc5195735a6fbe55b63 NOW.


Creating pystan model & assigning it to variable name "exponential_cov_measure_error".
Stan options:
 {'model_name': 'exponential_cov_measure_error', 'include_paths': None, 'boost_lib': None, 'eigen_lib': None, 'verbose': False, 'obfuscate_model_name': True}
StanModel now available as variable "exponential_cov_measure_error"!
Compilation took a minute.


###  文件

In [8]:
StanModel_cache(exponential_cov_measure_error, 'exponential_cov_measure_error')

Model cached as:exponential_cov_measure_error.pkl


### 指数相关函数，不带测量误差

In [6]:
%%stan exponential_cov_no_error
functions {
  matrix cov_exponential(matrix distance_between, real sigma, real fai) {
    int N=rows(distance_between);
    matrix[N, N] cov;
    for (i in 1:N){
      for (j in 1:N){
        if (i==j)
          cov[i, j] = 1;
        else
          cov[i,j] = square(sigma) * exp(-1*fai*distance_between[i,j]);
      }
    }
    
    return cov;
  }
}

data {
  int<lower=1> D; //数据组数
  int<lower=1> N; //感知节点个数
  vector[N] recv_x;// 每组数据有N个感知节点
  vector[N] recv_y;// 
  real tran_power[D]; //信号源发送功率
  real tran_x[D]; //信号源坐标
  real tran_y[D];
  vector[N] tran_recv_distance[D];
  vector[N] Y[D];
 
}

transformed data {
  matrix[N, N] distance_between_recv;
  for (i in 1:N){
    for (j in 1:N){
      distance_between_recv[i,j] = sqrt((recv_x[i]-recv_x[j])^2 + (recv_y[i]-recv_y[j])^2);
    }
  }
}

parameters {
  real gama; // 自由空间传播损耗因子
  real fai;  // matern 协方差函数的距离 参数
  real sigma; // 空间方差  
 // real sigma_eta; // 测量误差
}
transformed parameters {

}

model {
  matrix[N,N] bigSigma;
  //vector[N] i_vec = rep_vector(1.0, N);
  //matrix[N, N] unit_matrix = diag_matrix(i_vec);
  
  /********************/
  /*    指定先验分布    */
  /********************/
  gama ~ uniform(1,10);
  fai ~ normal(20, 100);
  sigma ~ inv_gamma(2, 10);
  //sigma_eta ~ inv_gamma(2, 1);
 
  
  
  bigSigma = cov_exponential(distance_between_recv, sigma, fai); //+ square(sigma_eta)*unit_matrix;
  for (i in 1:D){
    Y[i] ~ multi_normal(tran_power[i] - 10 * gama * log10(tran_recv_distance[i]), bigSigma); 
  }
   
}

INFO:pystan:COMPILING THE C++ CODE FOR MODEL exponential_cov_no_error_deb3c9acd90a80ff01f1be975e9efa7e NOW.


Creating pystan model & assigning it to variable name "exponential_cov_no_error".
Stan options:
 {'model_name': 'exponential_cov_no_error', 'include_paths': None, 'boost_lib': None, 'eigen_lib': None, 'verbose': False, 'obfuscate_model_name': True}
StanModel now available as variable "exponential_cov_no_error"!
Compilation took a minute.


In [7]:
StanModel_cache(exponential_cov_no_error, 'exponential_cov_no_error')

Model cached as:exponential_cov_no_error.pkl
